In [ ]:
import pandas as pd
from prophet import Prophet
from datetime import date, timedelta, datetime
import plotly.express as px

In [ ]:
confirmed_global_df = pd.read_csv('time_series_covid19_confirmed_global.csv')
confirmed_global_df.head()

In [ ]:
country_main = 'Germany'
confirmed_global_df = confirmed_global_df[confirmed_global_df['Country/Region'] == country_main].reset_index(drop=True)
confirmed_global_df

In [ ]:
confirmed_global_df.drop(['Province/State', 'Lat', 'Long'], axis=1, inplace=True)
df = pd.melt(confirmed_global_df, id_vars=['Country/Region'], value_vars=confirmed_global_df.columns[1:])
df.drop(['Country/Region'], axis=1, inplace=True)

In [ ]:
#convert the values to contain new cases per day
df['ds'] = df['variable'].apply(lambda x: datetime.strptime(str(x), "%m/%d/%y"))
df.drop(['variable'], axis=1, inplace=True)
df['value'] = df['value'].diff()
df.loc[0,'value'] = 0
df.columns = ['y', 'ds']

# Training

In [ ]:
m = Prophet(interval_width=0.95, daily_seasonality=True)
m.fit(df)

In [ ]:
future = m.make_future_dataframe(periods=365,freq='D')
forecast = m.predict(future)
forecast.head()

In [ ]:
plot1 = m.plot(forecast)

In [ ]:
# Seasonality by timeframe
plt2 = m.plot_components(forecast)

In [ ]:
from prophet.plot import add_changepoints_to_plot
fig = m.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), m, forecast)

# Holidays

In [ ]:
holidays_df = pd.read_csv('holidays_df_of_70_countries_for_covid_19_2022_Temporary.csv')
germany_holidays = holidays_df[holidays_df['country'] == country_main]

In [ ]:
germany_holidays.drop(['ds', 'country', 'code', 'country_official_name', 'prior_scale', 'source'], axis=1, inplace=True)

In [ ]:
germany_holidays.columns= ['ds', 'holiday', 'lower_window', 'upper_window']
germany_holidays

In [ ]:
m = Prophet(holidays=germany_holidays)
m.add_country_holidays(country_name='DE')
forecast = m.fit(df).predict(future)

In [ ]:
fig = m.plot_components(forecast)